# Training Embeddings for Words

This tutorial shows how to learn **word embeddings** from co-occurrence statistics.

We use a simple NN architecture, a long with the conditional cost function used by the [Swivel](https://arxiv.org/pdf/1602.02215.pdf) algorithm. 

The learnt embeddings are then extracted from the model and saved as TSV file.

The following are the steps of this tutorial:


1. Define input data metadata
2. Implement data input function
3. Create feature columns
4. Create a custome estimator
5. Define the train and evaluate experiment
6. Set the experiment parameters
7. Run the experiment
8. Extract the learnt **word embeddings** from the model

<a href="https://colab.research.google.com/github/ksalama/data2cooc2emb2ann/blob/master/movie2ann/02-Training_Embeddings_for_Words.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Setup

In [ ]:
# !pip install -r ../requirements.txt

In [3]:
import os
import math
import numpy as np
import tensorflow as tf
from tensorflow import data
from datetime import datetime

In [7]:
WORKSPACE = './workspace'
COOC_DIR = '{}/cooc'.format(WORKSPACE)
MODELS_DIR = '{}/models'.format(WORKSPACE)
SEED = 19831060

In [9]:
!echo "Files:"
!ls {COOC_DIR}/
!echo ""

!echo "info:"
!head {COOC_DIR}/info.log
!echo ""

!echo "vocab file:"
!head {COOC_DIR}/vocab.txt
!echo ""

Files:
cooc-00000-of-00007.tfrecords cooc-00005-of-00007.tfrecords
cooc-00001-of-00007.tfrecords cooc-00006-of-00007.tfrecords
cooc-00002-of-00007.tfrecords info.log
cooc-00003-of-00007.tfrecords vocab.txt
cooc-00004-of-00007.tfrecords

info:
P: 1123186
N: 105886
min: -3.29281
max: 11.45451

vocab file:
sum
cocoa
introduced
ride
trespass
might
fabrics
thoris
protection
part



## 1.  Metadata

In [10]:
FEATURES_SCHEMA = {
    'item1': tf.FixedLenFeature(dtype=tf.string, shape=()),
    'item2': tf.FixedLenFeature(dtype=tf.string, shape=()),
    'score': tf.FixedLenFeature(dtype=tf.float32, shape=()),
    'weight': tf.FixedLenFeature(dtype=tf.float32, shape=()),
    'type': tf.FixedLenFeature(dtype=tf.string, shape=())
}

WEIGHT_FEATURE_NAME = 'weight'
TARGET_FEATURE_NAME = 'score'

## 2.  Data Input Function

In [11]:
def make_input_fn(file_pattern, 
                  batch_size=128, num_epochs=1, mode=tf.estimator.ModeKeys.EVAL):

    def _input_fn():
        dataset = tf.data.experimental.make_batched_features_dataset(
            file_pattern,
            batch_size,
            features=FEATURES_SCHEMA,
            label_key=TARGET_FEATURE_NAME,
            reader=tf.data.TFRecordDataset,
            shuffle_buffer_size=batch_size * 10,
            reader_num_threads=1,
            parser_num_threads=2,
            num_epochs=num_epochs,
            shuffle=(mode==tf.estimator.ModeKeys.TRAIN),
            sloppy_ordering=True,
            drop_final_batch=True
        )
        return dataset
    
    return _input_fn

## 3. Feature Columns

In [12]:
def create_feature_columns(embedding_size, vocab1_file, vocab2_file):
    
    feature_columns = []

    feature_columns.append(
        tf.feature_column.embedding_column(
            tf.feature_column.categorical_column_with_vocabulary_file(
                key='item1', 
                vocabulary_file=vocab1_file
            ), 
            embedding_size
        )
    )

    feature_columns.append(
        tf.feature_column.embedding_column(
            tf.feature_column.categorical_column_with_vocabulary_file(
                key='item2', 
                vocabulary_file=vocab2_file
            ), 
            embedding_size
        )
    )
        
    return feature_columns

## 4.  Custom Estimator

In [13]:
def compute_loss(labels, predictions, weights, types):
    
    def _positive_sample_cost(errors, weights):
        return 0.5 * weights * tf.math.square(errors)
    
    def _negative_sample_cost(errors, weights):
        return weights * tf.math.log(1 + tf.exp(errors))
    
    errors = predictions - labels
    
    p_loss = _positive_sample_cost(errors, weights)
    n_loss = _negative_sample_cost(errors, weights)
    loss = tf.where(tf.equal(types, 'P'), p_loss, n_loss)
    
    return tf.reduce_sum(loss)

def model_fn(features, labels, mode, params):
    
    items1 = features['item1']
    feature_columns = create_feature_columns(
        params.embedding_size, params.vocab1_file, params.vocab2_file)
    
    item1_layer = tf.feature_column.input_layer(
        features={'item1': items1}, feature_columns=[feature_columns[0]])
    
    if mode != tf.estimator.ModeKeys.PREDICT:
        items2 = features['item2']
        item2_layer = tf.feature_column.input_layer(
            features={'item2': items2}, feature_columns=[feature_columns[1]])
        
        dot_product = tf.keras.layers.Dot(axes=1)([item1_layer, item2_layer])
        logits = (params.max_value - params.min_value) * tf.sigmoid(dot_product) + params.min_value 

    predictions = None
    export_outputs = None
    loss = None
    train_op = None

    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions =  item1_layer
        export_outputs = {'predictions': tf.estimator.export.PredictOutput(predictions)}
    else:
        types = features['type']
        weights = features[WEIGHT_FEATURE_NAME]
        
#         loss = tf.losses.mean_squared_error(
#             labels=labels, 
#             predictions=tf.squeeze(logits),
#             weights=weights
#         )

        loss = compute_loss(
            labels=labels, 
            predictions=tf.squeeze(logits), 
            weights=weights, 
            types=types
        )
        
        train_op=tf.train.AdamOptimizer(params.learning_rate).minimize(
            loss=loss, global_step=tf.train.get_global_step())
        
    
    return tf.estimator.EstimatorSpec(
        mode=mode,
        predictions=predictions,
        export_outputs=export_outputs,
        loss=loss,
        train_op=train_op
    )


def create_estimator(params, run_config):
    
    estimator = tf.estimator.Estimator(
        model_fn,
        params=params,
        config=run_config
    )
    
    return estimator

## 5. Experiment

In [14]:
def run_experiment(params, run_config):
    
    train_data_files = params.train_data_files
    eval_data_files = params.eval_data_files
    
    # TrainSpec ####################################
    train_input_fn = make_input_fn(
        train_data_files,
        batch_size=params.batch_size,
        num_epochs=None,
        mode=tf.estimator.ModeKeys.TRAIN
    )
    
    train_spec = tf.estimator.TrainSpec(
        input_fn = train_input_fn,
        max_steps=params.traning_steps
    )
    ###############################################    
    
    # EvalSpec ####################################
    eval_input_fn = make_input_fn(
        eval_data_files,
        num_epochs=None,
        batch_size=params.batch_size,
    )

    eval_spec = tf.estimator.EvalSpec(
        name=datetime.utcnow().strftime("%H%M%S"),
        input_fn = eval_input_fn,
        steps=params.eval_steps,
        start_delay_secs=0,
        throttle_secs=params.eval_throttle_secs
    )
    ###############################################

    tf.logging.set_verbosity(tf.logging.INFO)
    
    if tf.gfile.Exists(run_config.model_dir):
        print("Removing previous artefacts...")
        tf.gfile.DeleteRecursively(run_config.model_dir)
            
    print("")
    estimator = create_estimator(params, run_config)
    print("")
    
    time_start = datetime.utcnow() 
    print("Experiment started at {}".format(time_start.strftime("%H:%M:%S")))
    print(".......................................") 

    tf.estimator.train_and_evaluate(
        estimator=estimator,
        train_spec=train_spec, 
        eval_spec=eval_spec
    )

    time_end = datetime.utcnow() 
    print(".......................................")
    print("Experiment finished at {}".format(time_end.strftime("%H:%M:%S")))
    print("")
    time_elapsed = time_end - time_start
    print("Experiment elapsed time: {} seconds".format(time_elapsed.total_seconds()))
    
    return estimator

## 6. Parameters 

In [15]:
MODEL_NAME = 'cooc2emb-01'
model_dir = os.path.join(MODELS_DIR, MODEL_NAME)
info_file = os.path.join(COOC_DIR, 'info.log')
min_value = 15
max_value = -5

info_map = {}

if os.path.exists(info_file):
    try:
        with open(info_file) as f:
            for line in f.readlines():
                key, value = line.split(":")
                info_map[key] = float(value)
        min_value = math.floor(info_map['min'])
        max_value = math.ceil(info_map['max'])
    except: pass
    
class HParams():
    pass

params  = HParams()
params.train_data_files = "{}/cooc-*.tfrecords".format(COOC_DIR)
params.eval_data_files = "{}/cooc-*.tfrecords".format(COOC_DIR)
params.vocab1_file = os.path.join(COOC_DIR,'vocab.txt')
params.vocab2_file = os.path.join(COOC_DIR,'vocab.txt')
params.embedding_size = 128
params.min_value = min_value
params.max_value = max_value
params.batch_size = 265
params.traning_steps = 30000
params.learning_rate = 0.001
params.eval_steps = 1
params.eval_throttle_secs = 0

print(vars(params))

run_config = tf.estimator.RunConfig(
    tf_random_seed=SEED,
    save_checkpoints_steps=1000,
    keep_checkpoint_max=3,
    model_dir=model_dir,
)

{'train_data_files': './workspace/cooc/cooc-*.tfrecords', 'eval_data_files': './workspace/cooc/cooc-*.tfrecords', 'vocab1_file': './workspace/cooc/vocab.txt', 'vocab2_file': './workspace/cooc/vocab.txt', 'embedding_size': 128, 'min_value': -4, 'max_value': 12, 'batch_size': 265, 'traning_steps': 30000, 'learning_rate': 0.001, 'eval_steps': 1, 'eval_throttle_secs': 0}


## 7. Run

In [16]:
estimator = run_experiment(params, run_config)


INFO:tensorflow:Using config: {'_model_dir': './workspace/models/cooc2emb-01', '_tf_random_seed': 19831060, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 3, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x12d45a4e0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}

Experiment started at 13:46:06
.......................................
INFO:tensorflow:Not using Distribute Coordina

INFO:tensorflow:Restoring parameters from ./workspace/models/cooc2emb-01/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [1/1]
INFO:tensorflow:Finished evaluation at 2019-09-28-14:46:15
INFO:tensorflow:Saving dict for global step 1000: global_step = 1000, loss = 153.97975
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1000: ./workspace/models/cooc2emb-01/model.ckpt-1000
INFO:tensorflow:global_step/sec: 52.267
INFO:tensorflow:loss = 350.5269, step = 1001 (1.915 sec)
INFO:tensorflow:global_step/sec: 161.445
INFO:tensorflow:loss = 422.83356, step = 1101 (0.628 sec)
INFO:tensorflow:global_step/sec: 166.753
INFO:tensorflow:loss = 598.6262, step = 1201 (0.589 sec)
INFO:tensorflow:global_step/sec: 173.807
INFO:tensorflow:loss = 493.87988, step = 1301 (0.575 sec)
INFO:tensorflow:global_step/sec: 218.464
INFO:tensorflow:loss = 388.08447, step = 1401 (0.458 sec)
INFO:tensorflow:global_step/sec: 193.51

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-09-28T14:46:40Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./workspace/models/cooc2emb-01/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [1/1]
INFO:tensorflow:Finished evaluation at 2019-09-28-14:46:40
INFO:tensorflow:Saving dict for global step 5000: global_step = 5000, loss = 150.1204
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: ./workspace/models/cooc2emb-01/model.ckpt-5000
INFO:tensorflow:global_step/sec: 62.5474
INFO:tensorflow:loss = 127.316734, step = 5001 (1.601 sec)
INFO:tensorflow:global_step/sec: 221.97
INFO:tensorflow:loss = 284.2875, step = 5101 (0.448 sec)
INFO:tensorflow:global_step/sec: 242.038
INFO:tensorflow:loss = 255.98628, step = 5201 (0.413 sec)
INFO:tensorflow:global_step/sec: 248.739
INFO:tensorflow:loss = 177.02512, step = 5301 (0.402 sec)
I

INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [1/1]
INFO:tensorflow:Finished evaluation at 2019-09-28-14:47:02
INFO:tensorflow:Saving dict for global step 9000: global_step = 9000, loss = 79.51082
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 9000: ./workspace/models/cooc2emb-01/model.ckpt-9000
INFO:tensorflow:global_step/sec: 76.5526
INFO:tensorflow:loss = 180.28967, step = 9001 (1.307 sec)
INFO:tensorflow:global_step/sec: 220.527
INFO:tensorflow:loss = 148.45726, step = 9101 (0.453 sec)
INFO:tensorflow:global_step/sec: 233.426
INFO:tensorflow:loss = 148.46855, step = 9201 (0.429 sec)
INFO:tensorflow:global_step/sec: 221.458
INFO:tensorflow:loss = 122.95598, step = 9301 (0.452 sec)
INFO:tensorflow:global_step/sec: 217.473
INFO:tensorflow:loss = 165.40877, step = 9401 (0.460 sec)
INFO:tensorflow:global_step/sec: 221.282
INFO:tensorflow:loss = 93.23185, step = 9501 (0.452 sec)
INFO:tensorflow:global_step/

INFO:tensorflow:Saving dict for global step 13000: global_step = 13000, loss = 60.493774
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 13000: ./workspace/models/cooc2emb-01/model.ckpt-13000
INFO:tensorflow:global_step/sec: 60.026
INFO:tensorflow:loss = 91.84486, step = 13001 (1.668 sec)
INFO:tensorflow:global_step/sec: 135.786
INFO:tensorflow:loss = 87.88068, step = 13101 (0.735 sec)
INFO:tensorflow:global_step/sec: 136.919
INFO:tensorflow:loss = 92.34535, step = 13201 (0.730 sec)
INFO:tensorflow:global_step/sec: 147.675
INFO:tensorflow:loss = 94.01558, step = 13301 (0.678 sec)
INFO:tensorflow:global_step/sec: 156.596
INFO:tensorflow:loss = 80.5742, step = 13401 (0.638 sec)
INFO:tensorflow:global_step/sec: 173.72
INFO:tensorflow:loss = 87.66031, step = 13501 (0.576 sec)
INFO:tensorflow:global_step/sec: 201.78
INFO:tensorflow:loss = 142.02843, step = 13601 (0.495 sec)
INFO:tensorflow:global_step/sec: 198.733
INFO:tensorflow:loss = 128.15382, step = 13701 (0.503 sec)
I

INFO:tensorflow:global_step/sec: 64.1871
INFO:tensorflow:loss = 76.23258, step = 17001 (1.557 sec)
INFO:tensorflow:global_step/sec: 161.743
INFO:tensorflow:loss = 122.71914, step = 17101 (0.619 sec)
INFO:tensorflow:global_step/sec: 189.06
INFO:tensorflow:loss = 76.01202, step = 17201 (0.528 sec)
INFO:tensorflow:global_step/sec: 189.888
INFO:tensorflow:loss = 65.13731, step = 17301 (0.527 sec)
INFO:tensorflow:global_step/sec: 243.39
INFO:tensorflow:loss = 100.023285, step = 17401 (0.410 sec)
INFO:tensorflow:global_step/sec: 228.643
INFO:tensorflow:loss = 72.787605, step = 17501 (0.437 sec)
INFO:tensorflow:global_step/sec: 248.518
INFO:tensorflow:loss = 62.92665, step = 17601 (0.402 sec)
INFO:tensorflow:global_step/sec: 254.714
INFO:tensorflow:loss = 86.24591, step = 17701 (0.393 sec)
INFO:tensorflow:global_step/sec: 240.943
INFO:tensorflow:loss = 55.37762, step = 17801 (0.416 sec)
INFO:tensorflow:global_step/sec: 210.782
INFO:tensorflow:loss = 92.25747, step = 17901 (0.474 sec)
INFO:ten

INFO:tensorflow:global_step/sec: 207.473
INFO:tensorflow:loss = 73.519775, step = 21201 (0.482 sec)
INFO:tensorflow:global_step/sec: 195.816
INFO:tensorflow:loss = 55.17125, step = 21301 (0.511 sec)
INFO:tensorflow:global_step/sec: 209.258
INFO:tensorflow:loss = 79.37118, step = 21401 (0.478 sec)
INFO:tensorflow:global_step/sec: 212.973
INFO:tensorflow:loss = 99.34381, step = 21501 (0.470 sec)
INFO:tensorflow:global_step/sec: 207.502
INFO:tensorflow:loss = 64.17811, step = 21601 (0.481 sec)
INFO:tensorflow:global_step/sec: 205.302
INFO:tensorflow:loss = 71.90171, step = 21701 (0.487 sec)
INFO:tensorflow:global_step/sec: 206.607
INFO:tensorflow:loss = 67.21979, step = 21801 (0.484 sec)
INFO:tensorflow:global_step/sec: 211.824
INFO:tensorflow:loss = 69.87035, step = 21901 (0.472 sec)
INFO:tensorflow:Saving checkpoints for 22000 into ./workspace/models/cooc2emb-01/model.ckpt.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:vocabulary_size = 4618 in item1 is inferred from the number of e

INFO:tensorflow:global_step/sec: 209.02
INFO:tensorflow:loss = 39.157948, step = 25401 (0.479 sec)
INFO:tensorflow:global_step/sec: 211.737
INFO:tensorflow:loss = 68.87771, step = 25501 (0.472 sec)
INFO:tensorflow:global_step/sec: 222.413
INFO:tensorflow:loss = 77.05466, step = 25601 (0.450 sec)
INFO:tensorflow:global_step/sec: 214.909
INFO:tensorflow:loss = 57.435314, step = 25701 (0.465 sec)
INFO:tensorflow:global_step/sec: 218.914
INFO:tensorflow:loss = 65.060844, step = 25801 (0.457 sec)
INFO:tensorflow:global_step/sec: 185.968
INFO:tensorflow:loss = 55.56697, step = 25901 (0.538 sec)
INFO:tensorflow:Saving checkpoints for 26000 into ./workspace/models/cooc2emb-01/model.ckpt.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:vocabulary_size = 4618 in item1 is inferred from the number of elements in the vocabulary_file ./workspace/cooc/vocab.txt.
INFO:tensorflow:vocabulary_size = 4618 in item2 is inferred from the number of elements in the vocabulary_file ./workspace/cooc/vocab.txt.

INFO:tensorflow:global_step/sec: 204.039
INFO:tensorflow:loss = 87.608284, step = 29601 (0.490 sec)
INFO:tensorflow:global_step/sec: 190.589
INFO:tensorflow:loss = 50.301926, step = 29701 (0.525 sec)
INFO:tensorflow:global_step/sec: 189.754
INFO:tensorflow:loss = 48.48297, step = 29801 (0.528 sec)
INFO:tensorflow:global_step/sec: 193.149
INFO:tensorflow:loss = 51.37583, step = 29901 (0.517 sec)
INFO:tensorflow:Saving checkpoints for 30000 into ./workspace/models/cooc2emb-01/model.ckpt.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:vocabulary_size = 4618 in item1 is inferred from the number of elements in the vocabulary_file ./workspace/cooc/vocab.txt.
INFO:tensorflow:vocabulary_size = 4618 in item2 is inferred from the number of elements in the vocabulary_file ./workspace/cooc/vocab.txt.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-09-28T14:49:06Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./workspace/models/

## 8. Extract Word embeddings

In [17]:
def extract_embeddings():
    
    with tf.Session() as sess:
        saver = tf.train.import_meta_graph(os.path.join(model_dir, 'model.ckpt-{}.meta'.format(params.traning_steps)))
        saver.restore(sess, os.path.join(model_dir, 'model.ckpt-{}'.format(params.traning_steps)))
        graph = tf.get_default_graph()
        weights_tensor = graph.get_tensor_by_name('input_layer_1/item2_embedding/embedding_weights:0')
        weights = np.array(sess.run(weights_tensor))

    return weights

In [18]:
embeddings = extract_embeddings()
print(len(embeddings))
print(embeddings[0])

INFO:tensorflow:Restoring parameters from ./workspace/models/cooc2emb-01/model.ckpt-30000
4618
[-1.62278153e-02  8.35192669e-03  6.37334958e-02 -7.49534443e-02
 -5.90263829e-02 -1.98351722e-02  2.44442914e-02  4.09853049e-02
 -1.63244009e-01 -1.00082673e-01  1.88681215e-01  1.36155501e-01
  4.47690189e-02  8.76321867e-02  1.43647134e-01  1.22811340e-01
  1.12567902e-01 -4.14417386e-02  8.52247477e-02 -4.53575291e-02
  1.25810191e-01  5.29873855e-02 -6.09087311e-02 -5.50278611e-02
 -3.59600633e-02 -9.22060683e-02 -2.23044003e-03  1.50493518e-01
 -1.47038862e-01 -2.06213906e-01 -7.73831382e-02 -3.29067148e-02
 -7.39928335e-02  1.62057698e-01 -3.79122123e-02  6.15024455e-02
  3.83931510e-02  2.13978380e-01  1.44554079e-01  5.28540574e-02
  2.23463103e-02  8.26468244e-02 -3.66577767e-02  3.64660248e-02
 -1.31757438e-01 -3.34521569e-02  8.68275166e-02  8.15316290e-03
 -1.66728690e-01  4.85628545e-02 -1.59075826e-01  1.51271209e-01
  5.37051633e-02  1.61711991e-01  5.98695874e-02  1.47865608

In [19]:
vocab_path = os.path.join(COOC_DIR,'vocab.txt')
output_path = os.path.join(WORKSPACE,'embeddings.tsv')

def write_embeddings_to_tsv():
    with open(output_path, 'w') as out_f:
        with open(vocab_path) as vocab_f:
            for index, item in enumerate(vocab_f):
                embedding = embeddings[index]
                print('\t'.join([item.strip()] + [str(x) for x in embedding]), file=out_f)
                
write_embeddings_to_tsv()

In [20]:
!head {output_path}

sum	-0.016227815	0.008351927	0.063733496	-0.074953444	-0.059026383	-0.019835172	0.024444291	0.040985305	-0.16324401	-0.10008267	0.18868122	0.1361555	0.04476902	0.08763219	0.14364713	0.12281134	0.1125679	-0.04144174	0.08522475	-0.04535753	0.12581019	0.052987386	-0.06090873	-0.05502786	-0.035960063	-0.09220607	-0.00223044	0.15049352	-0.14703886	-0.2062139	-0.07738314	-0.032906715	-0.07399283	0.1620577	-0.037912212	0.061502445	0.03839315	0.21397838	0.14455408	0.052854057	0.02234631	0.082646824	-0.036657777	0.036466025	-0.13175744	-0.033452157	0.08682752	0.008153163	-0.16672869	0.048562855	-0.15907583	0.15127121	0.053705163	0.16171199	0.059869587	0.14786561	0.21781012	-0.11023164	0.18690342	-0.18080953	-0.074919276	0.13120681	-0.0965925	-0.10500597	-0.070218496	-0.013330257	-0.13576055	0.00926273	0.038872655	0.094963245	-0.08598454	0.13208371	0.0009699633	-0.09308746	-0.07117708	-0.18194632	-0.12566215	0.12236518	0.059256975	-0.2142878	-0.0768036	-0.017037526	0.0502373	-0.062588446	-0.1621